<a href="https://colab.research.google.com/github/TheodoredaCunha/Homemade-GPT-Model/blob/main/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Loading and Cleaning the Dataset**

In [29]:
import string
import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
#getting all the filenames that contain the data
TOTAL_CHAPTERS  = 3
dataset = []
for i in range(1, TOTAL_CHAPTERS + 1):
  dataset.append("chapter{}.txt".format(i))

print(dataset)

['chapter1.txt', 'chapter2.txt', 'chapter3.txt']


In [39]:
from os import setpgid

PATH = "drive/MyDrive/gpt/"
VOCAB_SET = set()
VOCAB_DICTIONARY = {}
ALL_CHAPTERS = []
def cleanData(filepath): #lowercase all characters and removing punctuations
  with open(filepath) as f:
    full_chapter = f.read().lower()
    cleaned_chapter = full_chapter.translate (str.maketrans ('', '', string.punctuation)) #remove punctuations from every sentence
    cleaned_split_chapter = cleaned_chapter.split() #turn every sentence into a list of words
    chapter_vocab = set(cleaned_split_chapter) #update the set of unique wordds in this sentence
    ALL_CHAPTERS.append(cleaned_split_chapter) #append the cleaned chapter text to a single list
    VOCAB_SET.update(chapter_vocab) #update the set of unique words for all chapters in the data
    f.close() #close the file
 
def createVocabDictionary(): #fill out vocab_dictionary based on the vocab set
  vocab_list = list(VOCAB_SET)
  for count, word in enumerate(vocab_list): #assign a number to every word in the vocab list
    VOCAB_DICTIONARY[word] = count

'''
IMPLEMENTING THE FUNCTIONS:
'''
#opening and cleaning each chapter
for chapter in dataset:
  print(chapter)
  cleanData(PATH + chapter)

#creating the vocabulary dictionary
createVocabDictionary()

#convert each letter in the chapters into their assigned numbers from the dictionary
for chapter in ALL_CHAPTERS:
  for i in range(len(chapter)):
    chapter[i] = VOCAB_DICTIONARY[chapter[i]]

print(ALL_CHAPTERS[0])

chapter1.txt
chapter2.txt
chapter3.txt
[923, 576, 626, 859, 614, 274, 451, 178, 674, 1385, 1205, 1003, 1883, 1671, 230, 1385, 432, 1787, 1533, 1992, 1406, 441, 230, 1385, 207, 101, 2057, 1369, 2105, 1003, 1332, 363, 586, 579, 14, 618, 1312, 1348, 230, 426, 598, 1832, 705, 849, 1189, 923, 859, 1858, 207, 258, 614, 1199, 1422, 1738, 213, 927, 1084, 1271, 1229, 1858, 1199, 1334, 312, 1790, 705, 1975, 787, 621, 1899, 1229, 120, 1810, 1199, 1406, 573, 1209, 626, 859, 1858, 1509, 576, 64, 576, 1350, 791, 1768, 207, 1750, 1133, 614, 621, 927, 1428, 586, 1406, 1180, 321, 631, 1039, 1412, 441, 614, 1473, 1497, 600, 333, 1574, 1524, 1417, 196, 207, 238, 207, 1559, 1350, 1199, 427, 1440, 1738, 359, 576, 586, 555, 1678, 1913, 1858, 955, 444, 2058, 664, 207, 1559, 1350, 670, 230, 1281, 283, 230, 194, 1350, 1199, 1310, 576, 555, 1721, 87, 1858, 1671, 1851, 59, 1615, 2048, 230, 598, 868, 230, 1595, 856, 2048, 828, 1748, 1771, 1108, 1025, 207, 1908, 626, 1254, 1858, 626, 1559, 1489, 283, 230, 1942, 45

**Creating the GPT Model**

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [45]:
training = []
testing = []
sampling_rate = 0.1 #10% of the sentences from each chapter will be used as test data
for chapter in ALL_CHAPTERS:
  test_data_num = int(len(chapter)*0.1)
  training.append(chapter[test_data_num * -1:])
  testing.append(chapter[:test_data_num])  

for i in range(TOTAL_CHAPTERS):  
  training[i] = torch.tensor(training[i], dtype=torch.long)
  testing[i] = torch.tensor(testing[i], dtype=torch.long)